In [1]:
# load Libraries
import pandas as pd
import numpy as np
import re
import datetime as dt


In [2]:
# Load Dataset
df = pd.read_csv('/Users/mayanksinghrawat/Desktop/Projects/Retail_Analysis/Retail_Transaction_Dataset.csv')
df.head()

,CustomerID,ProductID,Quantity,Price,TransactionDate,PaymentMethod,StoreLocation,ProductCategory,DiscountApplied(%),TotalAmount
0,109318,C,7,80.079844,12/26/2023 12:32,Cash,"176 Andrew Cliffs\nBaileyfort, HI 93354",Books,18.677100,455.862764
1,993229,C,4,75.195229,8/5/2023 0:00,Cash,"11635 William Well Suite 809\nEast Kara, MT 19483",Home Decor,14.121365,258.306546
2,579675,A,8,31.528816,3/11/2024 18:51,Cash,"910 Mendez Ville Suite 909\nPort Lauraland, MO...",Books,15.943701,212.015651
3,799826,D,5,98.880218,10/27/2023 22:00,PayPal,"87522 Sharon Corners Suite 500\nLake Tammy, MO...",Books,6.686337,461.343769
4,121413,A,7,93.188512,12/22/2023 11:38,Cash,"0070 Michelle Island Suite 143\nHoland, VA 80142",Electronics,4.030096,626.030484


In [3]:
df.describe()

,CustomerID,Quantity,Price,DiscountApplied(%),TotalAmount
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,500463.982180,5.009290,55.067344,10.020155,248.334955
std,288460.917524,2.579808,25.971567,5.779534,184.554792
min,14.000000,1.000000,10.000430,0.000046,8.274825
25%,250693.750000,3.000000,32.549474,5.001013,95.163418
50%,499679.000000,5.000000,55.116789,10.030353,200.368393
75%,751104.750000,7.000000,77.456763,15.018367,362.009980
max,999997.000000,9.000000,99.999284,19.999585,896.141242


In [4]:
df.tail()

,CustomerID,ProductID,Quantity,Price,TransactionDate,PaymentMethod,StoreLocation,ProductCategory,DiscountApplied(%),TotalAmount
99995,726461,A,2,56.078258,7/17/2023 16:59,Credit Card,"3632 Darren Station Apt. 553\nEricaborough, RI...",Clothing,18.345145,91.581240
99996,328056,A,6,88.516406,5/30/2023 9:04,Credit Card,"821 Taylor Shoals\nEvansville, IL 70845",Electronics,3.995541,509.878179
99997,887304,B,4,72.385564,8/25/2023 7:59,Credit Card,"50653 Kara Lakes\nStephanieborough, RI 94492",Clothing,17.423979,239.092472
99998,326401,C,5,66.542239,2/5/2024 19:45,PayPal,"18756 Mcfarland Way Suite 866\nBarnettside, PR...",Electronics,14.345018,284.983717
99999,771566,C,5,38.087766,2/4/2024 11:53,Debit Card,"8046 Hull Drive\nPaulstad, GU 87218",Home Decor,2.966058,184.790305


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   CustomerID          100000 non-null  int64  
 1   ProductID           100000 non-null  object 
 2   Quantity            100000 non-null  int64  
 3   Price               100000 non-null  float64
 4   TransactionDate     100000 non-null  object 
 5   PaymentMethod       100000 non-null  object 
 6   StoreLocation       100000 non-null  object 
 7   ProductCategory     100000 non-null  object 
 8   DiscountApplied(%)  100000 non-null  float64
 9   TotalAmount         100000 non-null  float64
dtypes: float64(3), int64(2), object(5)
memory usage: 7.6+ MB


In [6]:
# Converting Transactiondate into datetime
df['TransactionDate']= pd.to_datetime(df['TransactionDate'])
df['TransactionDate'].dtype

dtype('<M8[ns]')

In [7]:
# Extract time-based features
df['Year']=df['TransactionDate'].dt.year
df['Month']=df['TransactionDate'].dt.month
df['Day']=df['TransactionDate'].dt.day
df['Weekday']=df['TransactionDate'].dt.day_name()
df['Hour']=df['TransactionDate'].dt.hour

In [8]:
# Clean and extract City and State from storelocation

# Split address into two lines
address_parts = df['StoreLocation'].str.split('\n', expand=True)

# Step 1: Extract standard format → e.g., Holand, VA 80142
standard_extract = address_parts[1].str.extract(
    r'(?P<City>[A-Za-z\s\.]+),\s*(?P<State>[A-Z]{2})\s*(?P<Zipcode>\d{5})'
)

# Step 2: Extract military format → e.g., FPO AE 09123 or APO AP 10928
military_extract = address_parts[1].str.extract(
    r'(?P<MilCity>[A-Z]PO),?\s*(?P<MilState>A[A-Z])\s*(?P<MilZipcode>\d{5})'
)

# Combine results: prioritize standard first, fallback to military
df['City'] = standard_extract['City'].combine_first(military_extract['MilCity'])
df['State'] = standard_extract['State'].combine_first(military_extract['MilState'])
df['Zipcode'] = standard_extract['Zipcode'].combine_first(military_extract['MilZipcode'])


In [9]:
df.head(7)

,CustomerID,ProductID,Quantity,Price,TransactionDate,PaymentMethod,StoreLocation,ProductCategory,DiscountApplied(%),TotalAmount,Year,Month,Day,Weekday,Hour,City,State,Zipcode
0,109318,C,7,80.079844,2023-12-26 12:32:00,Cash,"176 Andrew Cliffs\nBaileyfort, HI 93354",Books,18.677100,455.862764,2023,12,26,Tuesday,12,Baileyfort,HI,93354
1,993229,C,4,75.195229,2023-08-05 00:00:00,Cash,"11635 William Well Suite 809\nEast Kara, MT 19483",Home Decor,14.121365,258.306546,2023,8,5,Saturday,0,East Kara,MT,19483
2,579675,A,8,31.528816,2024-03-11 18:51:00,Cash,"910 Mendez Ville Suite 909\nPort Lauraland, MO...",Books,15.943701,212.015651,2024,3,11,Monday,18,Port Lauraland,MO,99563
3,799826,D,5,98.880218,2023-10-27 22:00:00,PayPal,"87522 Sharon Corners Suite 500\nLake Tammy, MO...",Books,6.686337,461.343769,2023,10,27,Friday,22,Lake Tammy,MO,76234
4,121413,A,7,93.188512,2023-12-22 11:38:00,Cash,"0070 Michelle Island Suite 143\nHoland, VA 80142",Electronics,4.030096,626.030484,2023,12,22,Friday,11,Holand,VA,80142
5,463050,D,3,54.093152,2023-08-15 04:24:00,Cash,"8492 Jonathan Drive\nNorth Robertshire, TN 67532",Electronics,10.888768,144.609223,2023,8,15,Tuesday,4,North Robertshire,TN,67532
6,888163,D,7,13.121937,2023-12-26 05:32:00,PayPal,USNV Harrell\nFPO AA 62814,Clothing,16.295127,76.885907,2023,12,26,Tuesday,5,FPO,AA,62814


In [10]:
# Calculating discounted price & revenue

df['DiscountedPrice'] = df['Price']*(1-df['DiscountApplied(%)']/100)
df['RevenueAfterDiscount'] = df['DiscountedPrice']*df['Quantity']

# which is our Totalamont column.

In [11]:
# Handling outliers using IQR
# For quantity
Q1 = df['Quantity'].quantile(0.25)
Q3 = df['Quantity'].quantile(0.75)
IQR = Q3-Q1

Lower_bound = Q1 - 1.5 * IQR
Upper_bound = Q3 + 1.5 * IQR

print(f"For Quantity: Q1 = {Q1}, Q3 = {Q3}, IQR = {IQR}")
 # just for knowledge to know the values 

# filetring
df = df[(df['Quantity'] >= Lower_bound) & (df['Quantity'] <= Upper_bound)]

# For Price

Q1 = df['Price'].quantile(0.25)
Q3 = df['Price'].quantile(0.75)
IQR = Q3 - Q1

Lower_bound = Q1 - 1.5 * IQR
Upper_bound = Q3 + 1.5 * IQR

print(f"For Price: Q1 = {Q1}, Q3 = {Q3}, IQR = {IQR}")

df = df[(df['Price'] >= Lower_bound) & (df['Price'] <= Upper_bound)]

# For TotalAmount

Q1 = df['TotalAmount'].quantile(0.25)
Q3 = df['TotalAmount'].quantile(0.75)
IQR = Q3 - Q1

Lower_bound = Q1 - 1.5 * IQR
Upper_bound = Q3 + 1.5 * IQR

print(f"For TotalAmount: Q1 = {Q1}, Q3 = {Q3}, IQR = {IQR}")

df = df[(df['TotalAmount'] >= Lower_bound) & (df['TotalAmount'] <= Upper_bound)]

For Quantity: Q1 = 3.0, Q3 = 7.0, IQR = 4.0
For Price: Q1 = 32.5494740525, Q3 = 77.4567634225, IQR = 44.90728937
For TotalAmount: Q1 = 95.1634184575, Q3 = 362.00997989999996, IQR = 266.84656144249993


In [12]:
# Reorder for readability
df = df[['CustomerID', 'ProductID', 'ProductCategory', 'Quantity', 'Price', 'DiscountApplied(%)',
         'DiscountedPrice', 'RevenueAfterDiscount', 'PaymentMethod', 'TransactionDate', 'Year', 'Month', 'Day',
         'Weekday', 'Hour', 'City', 'State','Zipcode']]


In [13]:
df.head(10)

,CustomerID,ProductID,ProductCategory,Quantity,Price,DiscountApplied(%),DiscountedPrice,RevenueAfterDiscount,PaymentMethod,TransactionDate,Year,Month,Day,Weekday,Hour,City,State,Zipcode
0,109318,C,Books,7,80.079844,18.677100,65.123252,455.862764,Cash,2023-12-26 12:32:00,2023,12,26,Tuesday,12,Baileyfort,HI,93354
1,993229,C,Home Decor,4,75.195229,14.121365,64.576637,258.306546,Cash,2023-08-05 00:00:00,2023,8,5,Saturday,0,East Kara,MT,19483
2,579675,A,Books,8,31.528816,15.943701,26.501956,212.015651,Cash,2024-03-11 18:51:00,2024,3,11,Monday,18,Port Lauraland,MO,99563
3,799826,D,Books,5,98.880218,6.686337,92.268754,461.343769,PayPal,2023-10-27 22:00:00,2023,10,27,Friday,22,Lake Tammy,MO,76234
4,121413,A,Electronics,7,93.188512,4.030096,89.432926,626.030484,Cash,2023-12-22 11:38:00,2023,12,22,Friday,11,Holand,VA,80142
5,463050,D,Electronics,3,54.093152,10.888768,48.203074,144.609223,Cash,2023-08-15 04:24:00,2023,8,15,Tuesday,4,North Robertshire,TN,67532
6,888163,D,Clothing,7,13.121937,16.295127,10.983701,76.885907,PayPal,2023-12-26 05:32:00,2023,12,26,Tuesday,5,FPO,AA,62814
7,843385,A,Home Decor,8,56.025164,6.344306,52.470757,419.766052,Debit Card,2023-10-11 06:48:00,2023,10,11,Wednesday,6,North Brettville,WV,72610
8,839609,B,Electronics,5,23.857981,18.703997,19.395585,96.977925,Credit Card,2024-02-27 11:13:00,2024,2,27,Tuesday,11,Cabreraborough,SD,17915
9,184135,D,Books,4,63.342777,7.640607,58.503004,234.012018,Debit Card,2023-11-05 01:46:00,2023,11,5,Sunday,1,Martinfurt,MO,75932


In [14]:
# Save clean file
df.to_csv('/Users/mayanksinghrawat/Desktop/Projects/Retail_Analysis/cleaned_data.csv', index = False)

In [ ]:
# RFM Analysis
# setting reference data as day after last transaction date
ref_date = df['TransactionDate'].max() + pd.Timedelta(days = 1)

# RFM metrics
RFM = df.groupby('CustomerID').agg({'TransactionDate': lambda x:(ref_date - x.max()).days,'CustomerID':'count', 'RevenueAfterDiscount': 'sum'})
RFM.columns = ['Recency', 'Frequency', 'Monetary']
RFM.reset_index(inplace=True) # after groupby customerid becomes our index

In [ ]:
# KMeans Clustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

scaler = StandardScaler()
RFM_scaled = scaler.fit_transform(RFM[['Recency', 'Frequency', 'Monetary']])

kmeans = KMeans(n_clusters=3,n_init = 'auto' , random_state=42)
RFM['Segment'] = kmeans.fit_predict(RFM_scaled)
RFM.groupby('Segment')[['Recency', 'Frequency', 'Monetary']].mean()


In [ ]:
segment_labels = {0: 'High-value', 1: 'At-risk', 2: 'New'}
RFM['SegmentLabel'] = RFM['Segment'].map(segment_labels)
RFM['SegmentLabel'].value_counts()


In [ ]:
RFM.head()

In [ ]:
# Churn Analysis

# Churn label
latest_transaction = df.groupby('CustomerID')['TransactionDate'].max().reset_index()
latest_transaction['DaysSinceLastPurchase'] = (ref_date - latest_transaction['TransactionDate']).dt.days
latest_transaction['Churn'] = latest_transaction['DaysSinceLastPurchase'].apply(lambda x: 1 if x > 60 else 0)

# Merge churn label with RFM
RFM_churn = RFM.merge(latest_transaction[['CustomerID', 'Churn']], on='CustomerID')

# Logistic Regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X = RFM_churn[['Recency', 'Frequency', 'Monetary']]
y = RFM_churn['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Logistic
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
Churn_log_pred = logreg.predict(X_test)

# Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
Churn_rf_pred = rf.predict(X_test)

# Reports
print("Logistic Regression:\n", classification_report(y_test, Churn_log_pred))
print("Random Forest:\n", classification_report(y_test, Churn_rf_pred))


In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, Churn_log_pred))
